# Uniform priors in `easyCore`

The use of {term}`Bayesian methods` is becoming more and more popular in the analysis of model-dependent data, in particular in neutron scattering.
Bayesian methods is the continuation of the [](./prob_data.ipynb) section, where we thought about our data as a probability distribution. 
In this section, we will also consider how we can use our {term}`prior knowledge` about the system we are studying in the analysis. 

Before looking at anything new, we will generate the same simulated data as in the [](./fitting.ipynb) section. 

In [6]:
from easyCore import np

np.random.seed(123)

a_true = -0.9594
b_true = 7.294
c_true = 3.102

N = 50
x = np.sort(10 * np.random.rand(N))
yerr = 0.1 + 3 * np.random.rand(N)
y = a_true * x ** 2 + b_true * x + c_true
y += np.abs(y) * 0.2 * np.random.randn(N)

However, now we want to integrate our {term}`prior knowledge`, the most common way to do this is to have bounds on our parameters. 
By having these bounds we are saying that the probability of our parameter being less than some lower bound or greater then some upper bound is zero. 
For example, if parameter `b` in our quadratic model has bounds of 0 and 10, then there is an equal probability that the value of `b` can be anything in between 0 and 10, i.e., it has a uniform prior probability distribution. 


```{figure} ./images/uniform.png
---
height: 250px
name: uniform
---
A uniform distribution (blue line), from 0 to 10.
```

We can include information about the upper and lower bound of the fitting in our `easyCore` parameters. 

In [8]:
from easyCore.Objects.Variable import Parameter

a = Parameter(name='a', value=a_true, fixed=False, min=-5.0, max=0.5)
b = Parameter(name='b', value=b_true, fixed=False, min=0, max=10)
c = Parameter(name='c', value=c_true, fixed=False, min=-20, max=50)

We can then perform the analysis in the same fashion as previously, however, this time the bounds will be respected. 

In [9]:
from easyCore.Objects.ObjectClasses import BaseObj
from easyCore.Fitting.Fitting import Fitter

def math_model(x, *args, **kwargs):
    return a.raw_value * x ** 2 + b.raw_value * x + c.raw_value

quad = BaseObj(name='quad', a=a, b=b, c=c)
f = Fitter(quad, math_model)

res = f.fit(x=x, y=y, weights=yerr)

a, b, c

(<Parameter 'a': -0.86+/-0.08, bounds=[-5.0:0.5]>,
 <Parameter 'b': 6.4+/-0.8, bounds=[0:10]>,
 <Parameter 'c': 4.2+/-1.9, bounds=[-20:50]>)

Currently `easyCore` only supports uniform prior probability distributions, however, there is plans to extend this in future. 